# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dzhetygara,52.3544,61.0138,-7.64,83,30,4.72,KZ,1731299065
1,1,thompson,63.5032,-91.0725,-17.53,95,95,5.15,CA,1731299071
2,2,suva,-18.9787,177.7857,25.47,81,7,9.60,FJ,1731299077
3,3,st. john's,49.3207,-51.5476,3.39,70,26,13.00,CA,1731299078
4,4,coyhaique,-46.7493,-72.2080,-4.36,97,99,3.27,CL,1731299084


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    alpha=0.4,
    size="Humidity",
    color="City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

For the cell below, I narrowed down the results to include the cities with a minumum temperature of 18 C (65 F), a maximum temperature of 37 C (100 F), a wind speed of 4.5 m/s, and cloud-cover of no more than 20%.

In [40]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_df = city_data_df[city_data_df['Max Temp'] < 37]
narrow_df = narrow_df[narrow_df['Max Temp'] > 18]
narrow_df = narrow_df[narrow_df['Wind Speed'] < 4.5]
narrow_df = narrow_df[narrow_df['Cloudiness'] < 20]

# Drop any rows with null values
narrow_df.dropna(inplace=True)

# Display sample data
narrow_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,mulanje,-16.3368,36.2836,25.63,61,2,1.15,MW,1731299123
25,25,mhangura,-16.0040,30.0837,23.81,47,3,1.31,ZM,1731299150
42,42,hayma',18.4911,55.3629,22.58,66,4,2.36,OM,1731299199
77,77,debark',13.6945,37.5786,22.70,56,7,1.38,ET,1731299286
79,79,beohari,23.9563,81.7739,25.59,46,0,0.55,IN,1731299290
88,88,guankou,28.0390,113.9623,24.87,49,0,1.61,CN,1731299311
89,89,angaur state,7.2818,133.1383,28.77,69,9,0.48,PW,1731299312
105,105,lionel town,17.1924,-77.2930,28.47,73,14,2.66,JM,1731299347
109,109,mocimboa,-11.3325,40.5642,26.60,75,0,4.00,MZ,1731299353
127,127,penhalonga,-18.4779,33.0550,25.39,45,19,0.85,ZW,1731299403


### Step 3: Create a new DataFrame called `hotel_df`.

In [41]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
15,mulanje,MW,-16.3368,36.2836,61,
25,mhangura,ZM,-16.0040,30.0837,47,
42,hayma',OM,18.4911,55.3629,66,
77,debark',ET,13.6945,37.5786,56,
79,beohari,IN,23.9563,81.7739,46,
88,guankou,CN,28.0390,113.9623,49,
89,angaur state,PW,7.2818,133.1383,69,
105,lionel town,JM,17.1924,-77.2930,73,
109,mocimboa,MZ,-11.3325,40.5642,75,
127,penhalonga,ZW,-18.4779,33.0550,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [61]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mulanje - nearest hotel: No hotel found
mhangura - nearest hotel: No hotel found
hayma' - nearest hotel: No hotel found
debark' - nearest hotel: No hotel found
beohari - nearest hotel: No hotel found
guankou - nearest hotel: No hotel found
angaur state - nearest hotel: No hotel found
lionel town - nearest hotel: No hotel found
mocimboa - nearest hotel: No hotel found
penhalonga - nearest hotel: No hotel found
las lomitas - nearest hotel: No hotel found
paso de los libres - nearest hotel: No hotel found
zambezi - nearest hotel: No hotel found
ziniare - nearest hotel: No hotel found
ikalamavony - nearest hotel: No hotel found
doka - nearest hotel: No hotel found
launceston - nearest hotel: Pub in the Paddock
bhamo - nearest hotel: No hotel found
sitia - nearest hotel: No hotel found
eydhafushi - nearest hotel: No hotel found
tarija - nearest hotel: Hotel San Javier
montepuez - nearest hotel: No hotel found
yanbu - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
15,mulanje,MW,-16.3368,36.2836,61,No hotel found
25,mhangura,ZM,-16.0040,30.0837,47,No hotel found
42,hayma',OM,18.4911,55.3629,66,No hotel found
77,debark',ET,13.6945,37.5786,56,No hotel found
79,beohari,IN,23.9563,81.7739,46,No hotel found
88,guankou,CN,28.0390,113.9623,49,No hotel found
89,angaur state,PW,7.2818,133.1383,69,No hotel found
105,lionel town,JM,17.1924,-77.2930,73,No hotel found
109,mocimboa,MZ,-11.3325,40.5642,75,No hotel found
127,penhalonga,ZW,-18.4779,33.0550,45,No hotel found


There were only 2 nearby hotels found in the entire dataframe.

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [70]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    width=1200,
    height=600,
    geo=True,
    tiles="OSM",
    alpha=0.4,
    color="City",
    size="Humidity",
    hover_cols=["Hotel Name", "Country"]
)


# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)